### Imports

In [1]:
import ast
import sys
import json
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm.notebook import tqdm
from os.path import basename, dirname, isdir, isfile, join

sys.path.append("../setup/")
from gen_data import *

### Setup model

In [2]:
# model = keras.models.load_model("/Volumes/hd_4tb/results/runs/run_2/2_nonlinear.h5")
model_path = "/Users/pstetz/Desktop/confidential/.project/run/lgbm/3_full_data.pkl"
with open(model_path, "rb") as f:
    model = pickle.load(f)
    
with open("/Users/pstetz/Desktop/confidential/.project/summary/norm.json", "r") as f:
    norm_info = json.load(f)
    
with open ("/Users/pstetz/Desktop/confidential/.project/summary/info_order.json", "r") as f:
    order = json.load(f)
    
norm_info = {order[k]: v for k, v in norm_info.items()}

### Setup data

In [3]:
def _grey_mean_std(stats_file):
    with open(stats_file, "r") as f:
        data = json.load(f)
        data = ast.literal_eval(data)
        data = ast.literal_eval(data)
    means, stds = [], []
    for k, v in data.items():
        means.append(v["mean"])
        stds.append(v["std"])
    mn = sum(means) / len(means)
    std = (sum([e**2 for e in stds]) / len(means)) ** 0.5
    return mn, std

root = "/Users/pstetz/Desktop/confidential/.project"
df = pd.read_csv(join(root, "summary/model_input.csv"))
onset_df = pd.read_csv(join(root, "interpolate/raw/conn152/gonogo/onsets.csv"))
gvol = nib.load(join(root, "interpolate/raw/conn152/structural/gm_probseg.nii.gz")).get_fdata()
grey_mean, grey_std = _grey_mean_std("/Users/pstetz/Desktop/confidential/.project/summary/grey_norm.json")
gvol[:, :, :] = np.divide(np.subtract(gvol[:, :, :], grey_mean), grey_std)

mask_dir = "/Users/pstetz/Desktop/confidential/.project/masks"
available_volumes = np.load("../setup/available_volumes.npy")
masks = load_masks(mask_dir)
train_cols = [c for c in df.columns if c.startswith("is_")] + ["age"]

### Helper functions

In [4]:
def _inbetween_file(path1, path2):
    """
    z --> indicates half step forwards
    a --> indicates half step back
    s --> indicates a stop
    """
    name1, name2 = basename(path1), basename(path2)
    directory = dirname(path1)
    if not name1.endswith("s.npy") and not name2.endswith("s.npy"):
        return join(directory, name1.replace(".npy", "_zs.npy"))
    elif len(name1) > len(name2):
        return join(directory, name1.replace("s.npy", "zs.npy"))
    return join(directory, name2.replace("s.npy", "as.npy"))

def _get_time(path, TR=2):
    name = basename(path).replace("s.npy", "").replace(".npy", "")
    if "_" not in name:
        return TR * int(name)
    _time = int(name.split("_")[0]) * TR
    for i, c in name.split("_")[1]:
        if c == "a":
            _time -= 1 / 2**(i+1)
        elif c == "z":
            _time += 1 / 2**(i+1)
        else: raise
    return _time

def norm_features(x, norm_info):
    for feat in x.columns:
        if feat in ['Go', 'Target', 'Anger', 'Disgust', 'Neutral', '1', 'Happy', 'Baseline', 'NoGo', 'Sad', '6', 'Fear', 'NonTarget']:
            x[feat] = np.clip(x[feat], 0, 15)
        mn = norm_info[feat]["mean"]
        std = norm_info[feat]["std"]
        x[feat] = np.divide(np.subtract(x[feat], mn), std)
    return x

def mean_activation(masks, pvol, nvol, gvol):
    activations = dict()
    for label, volume in [("prev", pvol), ("next", nvol)]:
        for mask in masks:
            code, data = mask["code"], mask["data"]
            region = np.multiply(data, volume)                                                                                                                                                                                                                                                                                                                    
            activations["mean_%s_%s" % (code, label)] = np.mean( np.multiply(gvol, region) )
    return activations

def at_most_level(num, filepath):
    name = basename(filepath)
    t = int(name.split(".")[0].split("_")[0])
    if t > 50:
        return False
    if "_" not in name:
        return 0 <= num
    return len(name.split("_")[1].replace("s.npy", "")) <= num

def guess_volume(
        pvol, nvol, gvol, t_index, stable_x,
        model, masks, norm_info,
    ):
    mask = nib.load("/Users/pstetz/Desktop/confidential/.project/interpolate/MNI152_T1_2mm_brain_mask.nii.gz").get_fdata()
    volume = np.zeros(shape=pvol.shape)
    x_n, y_n, z_n = volume.shape
    stable_x = stable_x.append([stable_x] * (z_n - 1), ignore_index=True)
    for i in tqdm(range(40, 50)):
        for j in range(60, 70):
            x = stable_x.copy()
            x["x"], x["y"], x["t"] = i, j, t_index
            x["z"] = list(range(z_n)) # FIXME: double check that this is correct for z
            for m in masks:
                x["in_%s" % m["code"]] = [int(bool(m["data"][i, j, k])) for k in range(z_n)]
            norm_x = norm_features(x, norm_info)
            for label, data in [("prev", pvol), ("next", nvol), ("grey", gvol)]:
                norm_x[label] = data[i, j, :]
            return norm_x
            volume[i, j, :] = model.predict(norm_x)
    return np.multiply(volume, mask)

### Main

Rerunning will decrease the TR by 1/2

In [7]:
columns = pd.read_csv("/Users/pstetz/Desktop/confidential/.project/features.csv")
columns = [c for c in columns.columns if c != "pred"]
volume[columns]

,Go,Target,Anger,Disgust,Neutral,1,Happy,Baseline,NoGo,Sad,...,mean_911981_prev,mean_954861_next,mean_954861_prev,t,x,y,z,prev,next,grey
0,0.632366,0.368107,0.459208,0.428577,0.460187,0.793321,0.463731,0.39187,0.565595,0.45869,...,2.73837,-0.662927,-0.579665,-1.20591,-0.286953,0.385377,-2.610688,0.0,0.0,-0.273702
1,0.632366,0.368107,0.459208,0.428577,0.460187,0.793321,0.463731,0.39187,0.565595,0.45869,...,2.73837,-0.662927,-0.579665,-1.20591,-0.286953,0.385377,-2.546925,0.0,0.0,-0.273702
2,0.632366,0.368107,0.459208,0.428577,0.460187,0.793321,0.463731,0.39187,0.565595,0.45869,...,2.73837,-0.662927,-0.579665,-1.20591,-0.286953,0.385377,-2.483162,0.0,0.0,-0.273702
3,0.632366,0.368107,0.459208,0.428577,0.460187,0.793321,0.463731,0.39187,0.565595,0.45869,...,2.73837,-0.662927,-0.579665,-1.20591,-0.286953,0.385377,-2.419399,0.0,0.0,-0.273702
4,0.632366,0.368107,0.459208,0.428577,0.460187,0.793321,0.463731,0.39187,0.565595,0.45869,...,2.73837,-0.662927,-0.579665,-1.20591,-0.286953,0.385377,-2.355637,0.0,0.0,-0.273702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,0.632366,0.368107,0.459208,0.428577,0.460187,0.793321,0.463731,0.39187,0.565595,0.45869,...,2.73837,-0.662927,-0.579665,-1.20591,-0.286953,0.385377,2.872914,0.0,0.0,-0.273702
87,0.632366,0.368107,0.459208,0.428577,0.460187,0.793321,0.463731,0.39187,0.565595,0.45869,...,2.73837,-0.662927,-0.579665,-1.20591,-0.286953,0.385377,2.936677,0.0,0.0,-0.273702
88,0.632366,0.368107,0.459208,0.428577,0.460187,0.793321,0.463731,0.39187,0.565595,0.45869,...,2.73837,-0.662927,-0.579665,-1.20591,-0.286953,0.385377,3.000440,0.0,0.0,-0.273702
89,0.632366,0.368107,0.459208,0.428577,0.460187,0.793321,0.463731,0.39187,0.565595,0.45869,...,2.73837,-0.662927,-0.579665,-1.20591,-0.286953,0.385377,3.064202,0.0,0.0,-0.273702


In [6]:
volume_dir = join(root, "interpolate/volumes")

record = 291 # CONN152 gonogo
TR = 2 # seconds
subject_df = pd.DataFrame(df.iloc[record][train_cols]).T.reset_index(drop=True)

for lvl in range(4):
    files = list(sorted(glob(join(volume_dir, "*"))))
    files = [f for f in files if at_most_level(lvl, f)]
    for i in range(len(files) - 1):
        file1, file2 = files[i], files[i + 1]
        dst = _inbetween_file(file1, file2)
        if isfile(dst):
            print("%s already exists.  Skipping" % dst)
        print("Interpolating between files \n\t%s\n\t%s \n\nto create\n\t%s" % (file1, file2, dst))
        print("")
        pvol = np.load(file1)
        nvol = np.load(file2)
        t = (_get_time(file1, TR=TR) + _get_time(file2, TR=TR)) / 2

        onsets = last_onset(onset_df, "gonogo", t).reset_index(drop=True)
        mask_activations = mean_activation(masks, pvol, nvol, gvol)
        stable_x = pd.concat([subject_df, onsets], axis=1)
        for k, v in mask_activations.items():
            stable_x[k] = v
        for feat in stable_x.columns:
            stable_x[feat] = stable_x[feat].astype(np.float64)

        volume = guess_volume(pvol, nvol, gvol, t * TR, stable_x, model, masks, norm_info)
        raise
        np.save(dst, volume)

%s already exists.  Skipping
Interpolating between files 
	/Users/pstetz/Desktop/confidential/.project/interpolate/volumes/000.npy
	/Users/pstetz/Desktop/confidential/.project/interpolate/volumes/001.npy 

to create
	/Users/pstetz/Desktop/confidential/.project/interpolate/volumes/000_zs.npy



RuntimeError: No active exception to reraise

### Initialize volumes

Run this first before everything else

In [3]:
def _init_func(filepath, output_dir):
    data = nib.load(filepath).get_fdata()
    N = data.shape[3]
    for i in tqdm(range(N)):
        volume = data[:, :, :, i]
        dst = join(output_dir, "%03d.npy" % i)
        if isfile(dst):
            print("%s already exists" % dst)
            continue
        np.save(dst, volume)

raw = "/Users/pstetz/Desktop/confidential/.project/interpolate/raw/conn152"
dst_dir = "/Users/pstetz/Desktop/confidential/.project/interpolate/volumes_290"

task_dir = join(raw, "conscious")
func = join(task_dir, "normalized.nii.gz")
_init_func(func, dst_dir)